## Notebook to ensure tip order is correct

#### Concerns: 
    
1. Are the names being swapped correctly when changing to msprime naming (1-ntips) and then back to the names on the species tree (toytree)?
2. Is the sequence array being ordered correctly given the names applied to the trees? 
3. For balanced splits (cherries) is arbitrary ordering of the tree consequential?
    
    
#### How each is handled by ipcoal:
1. The sptree is loaded by toytree and ladderized which deterministically will return the same name ordering any time this tree is loaded. A dictionary is then created (.tipdict) that stores a mapping of mstip-integers to sptree tip-labels starting from 1-ntips. The order of tips in the ...Msprime will return trees with tips labeled from 1-nsamples where, e.g., the numbers 1-8 represent populations 1-8 as defined in demography. 

2. 
3. 
    
    
#### How this is demonstrated in this notebook:  
1.  
2.  
3. ...maybe parsing and rooting of trees by toytree is not quite right for balanced tree edge rooting... needs *another* look.
    


In [1]:
import ipcoal
import toytree
import toyplot
import numpy as np

### 1. Name ordering

We can see clearly in this example that the name ordering is working correctly. Here I sample three individuals from each population in a tree with 8 populations. I then set the Ne to be large or small for one of each pair of sister species. We can see that the odd-numbered tips have large Ne, and the even-numbered tips have small Ne and thus short coalescent times. The relationships and coalescent times appear as expected. 

#### imbalanced tree

This looks correct.

In [7]:
# load a species tree
tree = toytree.rtree.imbtree(8, treeheight=1e6)

# set Ne values to high for odd number tip names
tree = tree.set_node_values(
    attr="Ne",
    default=1e3,
    values={7: 2e5, 5:2e5, 3:2e5, 0:2e5},
)

# load the ipcoal model and sample trees
model = ipcoal.Model(tree, samples=3, seed=123)
model.sim_loci(3);

# plot the species tree and a sampled genealogy side by side
canvas = toyplot.Canvas(width=700, height=400)
ax0 = canvas.cartesian(bounds=("10%", "45%", "10%", "90%"))
ax1 = canvas.cartesian(bounds=("55%", "90%", "10%", "90%"))
ax0.show = False; ax1.show = False

# draw species tree
tre = toytree.tree(model.tree)
tre.draw(
    axes=ax0, 
    node_labels=tre.get_node_values("idx", 1, 1),
    edge_widths="Ne",
);

# draw genealogy
tre = toytree.tree(model.df.genealogy[0])
tre.draw(axes=ax1);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="ta3ba5c9be7ba431ebee38a692881e83a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 400.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1 r0 r2 r3 r4 r5 r6 r7 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 r0-1 r0-0 r0-2 r1-0 r1-2 r1-1 r2-2 r2-0 r2-1 r3-0 r3-2 r3-1 r4-1 r4-0 r4-2 r5-1 r5-0 r5-2 r6-1 r6-0 r6-2 r7-2 r7-0 r7-1

#### balanced tree

In [8]:
# load a species tree
tree = toytree.rtree.baltree(8, treeheight=1e6)

# set Ne values to high for odd number tip names
tree = tree.set_node_values(
    attr="Ne",
    default=1e3,
    values={7: 2e5, 5:2e5, 3:2e5, 1:2e5},
)

# load the ipcoal model and sample trees
model = ipcoal.Model(tree, samples=3, seed=123)
model.sim_loci(3);

# plot the species tree and a sampled genealogy side by side
canvas = toyplot.Canvas(width=700, height=400)
ax0 = canvas.cartesian(bounds=("10%", "45%", "10%", "90%"))
ax1 = canvas.cartesian(bounds=("55%", "90%", "10%", "90%"))
ax0.show = False; ax1.show = False

# draw species tree
tre = toytree.tree(model.tree)
tre.draw(
    axes=ax0, 
    node_labels=tre.get_node_values("idx", 1, 1),
    edge_widths="Ne",
);

# draw genealogy
tre = toytree.tree(model.df.genealogy[0])
tre.draw(axes=ax1);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t0799575298b149eda7b240506b6b836b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 400.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 r0-1 r0-0 r0-2 r1-0 r1-2 r1-1 r3-2 r3-1 r3-0 r2-1 r2-0 r2-2 r7-2 r7-0 r7-1 r6-1 r6-0 r6-2 r5-1 r5-0 r5-2 r4-2 r4-0 r4-1

### (2) Are the sequences ordered correctly?

This looks like no. And I'm starting to worry that the there is an error either in rooting, or in evolving data along an edge near the root, since this edge length is basically missing...

#### imbalanced tree

The edge lengths look very good, but the tip names are wrong.

In [25]:
# load a species tree
tree = toytree.rtree.imbtree(8, treeheight=1e6)

# set Ne values on tips
tree = tree.set_node_values(
    attr="Ne",
    default=1e3,
    values={7: 2e5, 5:2e5, 3:2e5, 0:2e5},
)

# load the ipcoal model and sample trees
model = ipcoal.Model(tree, samples=3, seed=123, recomb=0)
model.sim_loci(1, 100000)
model.infer_gene_trees()

# plot the species tree and a sampled genealogy side by side
canvas = toyplot.Canvas(width=700, height=400)
ax0 = canvas.cartesian(bounds=("10%", "45%", "10%", "90%"))
ax1 = canvas.cartesian(bounds=("55%", "90%", "10%", "90%"))
ax0.show = False; ax1.show = False

# draw genealogy
toytree.tree(model.df.genealogy[0]).draw(axes=ax0);

# draw inferred tree
tre = toytree.tree(model.df.inferred_tree[0])
rtre = tre.root(wildcard="r7", edge_features='dist')
rtre.draw(axes=ax1);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t64fed3ddf4854b348c6c83a7f6eb4a13" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 400.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1-2 r1-1 r1-0 r0-1 r0-0 r0-2 r2-2 r2-1 r2-0 r3-1 r3-2 r3-0 r4-2 r4-0 r4-1 r5-1 r5-0 r5-2 r6-2 r6-1 r6-0 r7-2 r7-0 r7-1 r0-0 r0-1 r0-2 r1-1 r1-2 r1-0 r2-2 r2-1 r2-0 r3-1 r3-0 r3-2 r4-0 r4-2 r4-1 r5-1 r5-0 r5-2 r6-1 r6-0 r6-2 r7-2 r7-0 r7-1

#### balanced tree

Rooting edge length problem here needs fixed in toytree... 

In [23]:
# load a species tree
tree = toytree.rtree.baltree(8, treeheight=1e6)

# set Ne values on tips
tree = tree.set_node_values(
    attr="Ne",
    default=1e3,
    values={7: 2e5, 5:2e5, 3:2e5, 1:2e5},
)

# load the ipcoal model and sample trees
model = ipcoal.Model(tree, samples=3, seed=123, recomb=0)
model.sim_loci(1, 100000)
model.infer_gene_trees()

# plot the species tree and a sampled genealogy side by side
canvas = toyplot.Canvas(width=700, height=400)
ax0 = canvas.cartesian(bounds=("10%", "45%", "10%", "90%"))
ax1 = canvas.cartesian(bounds=("55%", "90%", "10%", "90%"))
ax0.show = False; ax1.show = False

# draw genealogy
toytree.tree(model.df.genealogy[0]).draw(axes=ax0);

# draw inferred tree
tre = toytree.tree(model.df.inferred_tree[0])
rtre = tre.root(regex="r[4-7].")
rtre.draw(axes=ax1);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t0c33f4100c564b04a86dc0d8c50d92e7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 400.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0-2 r0-1 r0-0 r1-0 r1-2 r1-1 r3-1 r3-0 r3-2 r2-2 r2-1 r2-0 r7-2 r7-1 r7-0 r6-2 r6-0 r6-1 r4-2 r4-0 r4-1 r5-2 r5-0 r5-1 r0-2 r0-1 r0-0 r1-2 r1-1 r1-0 r2-2 r2-1 r2-0 r3-0 r3-1 r3-2 r4-0 r4-2 r4-1 r5-0 r5-2 r5-1 r6-2 r6-0 r6-1 r7-1 r7-2 r7-0